In [8]:
from pyspark.sql.functions import *

In [9]:
topic_read_stream = spark\
    .readStream\
    .format("kafka")\
    .option("kafka.bootstrap.servers","broker:9092")\
    .option("subscribe","commerce")\
    .option("startingOffsets","earliest")\
    .load()\
    .select(\
            col("key").cast("string"),\
            col("value").cast("string"),\
            "partition",\
            "timestamp")

topic_read_stream.isStreaming

True

In [10]:
topic_string_stream = topic_read_stream.select(col("key").cast("string"),col("value").cast("string"))

In [11]:
topic_string_stream_output = topic_string_stream.writeStream\
    .format("kafka")\
    .option("kafka.bootstrap.servers","broker:9092")\
    .option("topic","commerce_output")\
    .option("checkpointLocation","/usr/eric/kafka_checkpoint_kv_1")\
    .trigger(continuous="5 second")\
    .start()

In [27]:
topic_string_stream_output.status

{'message': 'Terminated with exception: Failed to construct kafka consumer',
 'isDataAvailable': False,
 'isTriggerActive': False}

In [28]:
topic_read2 = spark.read\
    .format("kafka")\
    .option("kafka.bootstrap.servers","localhost:9092")\
    .option("subscribe","commerce_output")\
    .load()

In [ ]:
topic_string2 = topic_read2.select(col("key").cast("string"),
                                   col("value").cast("string"))
topic_string2.show()